In [8]:
import bambi as bmb

from bambi.families.link import Link
from typing import Dict, Union

class Family:
    SUPPORTED_LINKS = [
        "cloglog",
        "identity",
        "inverse_squared",
        "inverse",
        "log",
        "logit",
        "probit",
        "softmax",
        "tan_2",
    ]

    def __init__(self, name, likelihood, link) :
        self.name = name
        self.likelihood = likelihood
        self.link = link
        self.aliases = {}

    @property
    def link(self):
        return self._link

    @link.setter
    def link(self, value):
        assert isinstance(value, dict), "Link functions must be specified with a 'dict'"
        links = {}
        for param_name, param_value in value.items():
            if isinstance(param_value, str):
                param_link = self.check_string_link(param_value)
            elif isinstance(value, Link):
                param_link = param_value
            else:
                raise ValueError(".link must be set to a string or a Link instance.")
            links[param_name] = param_link
        self._link = links

    def check_string_link(self, name):
        if not name in self.SUPPORTED_LINKS:
            raise ValueError(f"Link '{name}' cannot be used with family '{self.name}'")
        return Link(name)

    def set_alias(self, name, alias):
        """Set alias for an auxiliary variable of the family

        Parameters
        ----------
        name: str
            The name of the variable
        alias: str
            The new name for the variable
        """
        self.aliases.update({name: alias})

    def __str__(self):
        msg_list = [f"Response distribution: {self.likelihood.name}", f"Link: {self.link.name}"]
        if self.likelihood.priors:
            priors_msg = "\n  ".join([f"{k} ~ {v}" for k, v in self.likelihood.priors.items()])
            msg_list += [f"Priors:\n  {priors_msg}"]
        msg = "\n".join(msg_list)
        return msg

    def __repr__(self):
        return self.__str__()


In [21]:
from bambi.defaults import get_builtin_family

from bambi.families.univariate import Gaussian

from bambi.defaults.defaults import generate_likelihood

In [28]:
d = {"name": "Normal", "args": {"sigma": "HalfNormal"}, "parent": "mu"}
l = generate_likelihood(**d)
l

Likelihood(  
  name: Normal,
  parent: mu,
  priors: {'sigma': HalfNormal(sigma: 1)}
)

Likelihood(  
  name: Normal,
  parent: mu,
  priors: {'sigma': HalfNormal(sigma: 1)}
)

In [16]:
get_builtin_family("gaussian")

AssertionError: Link functions must be specified with a 'dict'

In [13]:
likelihood.parent

'mu'

In [12]:
likelihood.__dict__

{'name': 'Gaussian',
 'priors': {'sigma': HalfNormal(sigma: 1)},
 '_parent': 'mu',
 'dist': None}

In [9]:
sigma_prior = bmb.Prior("HalfNormal", sigma=1)
likelihood = bmb.Likelihood("Gaussian", parent="mu", sigma=sigma_prior)

In [10]:
link_dict = {"mu": "identity", "sigma": "log"}

In [11]:
Family("my_gaussian", likelihood, link_dict)

AttributeError: 'dict' object has no attribute 'name'

A tener en cuenta

* No solo es cambiar la interface de la formula
* Tambien es habilitar diferentes funciones de enlace para diferentes terminos


**Idea 1**

```python
link_dicts = {"mu": "identity", "sigma": "log"}
Model(... link=link_dicts)
```

La pregunta es, donde se almacenan estos `links`?

**Respuesta:** Tienen que ir a parar a las familias.

**Pregunta:** 

* El prior de los parametros auxiliares tiene que ir en el Likelihood???
* Empiezo a pensar que la respuesta es **NO**. Estamos mezclado dos cosas distintas.
    * Descripcion de la funcion de verosimilitud, sus parametros, y cual es el rol de cada uno. -> Esto siempre es necesario
    * Definicion de distribuciones a prior. -> Esto no es siempre necesario.
* Los Priors en la familia tambien son un **error**.
    * Un prior no forma parte de la descripcion de la familia.